In [1]:
import json
import ROOT
from analysis_framework import Dataset
from WWAnalysis import WWAnalysis
import os
from alt_setup_creator import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x853aea0


In [2]:
ROOT.EnableImplicitMT(12)

In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad

In [4]:
prod = True
no_rvec = False
output_collections = ""
dataset_path = "stage2-test.json"
template_par_plot_path = "plots/template_parametrisation/test"
histo_path = "data/histograms/test"
output_path = "data/snapshots/stage3/test"
output_meta = "stage3-test.json"
if prod:
    dataset_path = "stage2-full.json"
    template_par_plot_path = "plots/template_parametrisation/full"
    histo_path = "data/histograms/full"
    output_path = "data/snapshots/stage3/full"
    output_meta = "stage3-full.json"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = WWAnalysis(dataset)
analysis.init_categories()

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xb61d0e0


In [7]:
alt_setup_handler = AltSetupHandler("alt-configs.json", mirror=True)
alt_configs = alt_setup_handler.get_alt_setup()
alt_variations = alt_setup_handler.get_variations()

In [8]:
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]
analysis.set_signal_categories(signal_category)

True


In [9]:
print(json.dumps(analysis._categories, indent=2))

{
  "4f_sw_sl_signal": [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpR_signal",
    "4f_sw_sl_eRpL_signal"
  ],
  "4f_sl_bkg": [
    "4f_zz_sl_eLpR",
    "4f_zz_sl_eRpL",
    "4f_zznu_sl_eLpR",
    "4f_zznu_sl_eRpL",
    "4f_sze_sl_eLpL",
    "4f_sze_sl_eLpR",
    "4f_sze_sl_eRpR",
    "4f_sze_sl_eRpL",
    "4f_sznu_sl_eLpR",
    "4f_sznu_sl_eRpL",
    "4f_ww_sl_eLpR",
    "4f_ww_sl_eRpL",
    "4f_sw_sl_eLpL_bkg",
    "4f_sw_sl_eLpR_bkg",
    "4f_sw_sl_eRpR_bkg",
    "4f_sw_sl_eRpL_bkg"
  ],
  "4f_not_sl": [
    "4f_zz_h_eLpR",
    "4f_zz_h_eRpL",
    "4f_zz_l_eLpR",
    "4f_zz_l_eRpL",
    "4f_zzorww_h_eLpR",
    "4f_zzorww_l_eLpR",
    "4f_zzorww_l_eRpL",
    "4f_zznu_l_eLpR",
    "4f_zznu_l_eRpL",
    "4f_sznu_l_eLpR",
    "4f_sznu_l_eRpL",
    "4f_sze_l_eLpL",
    "4f_sze_l_eLpR",
    "4f_sze_l_eRpR",
    "4f_sze_l_eRpL",
    "4f_szeorsw_l_eLpL",
    "4f_szeorsw_l_eLpR",
    "4f_szeorsw_l_eRpR",
    "4f_szeorsw_l_eRpL",
    "4f_sw_l_eLpL",
    "4f_sw_l_

In [10]:
oo_setups = ["g1z_pos_1em05", "ka_pos_1em05", "la_pos_1em05", ]
analysis.define_optimal_observables(alt_setup_handler, only=oo_setups)

In [11]:
n_bins = 65
oo_configs = {
    "O_g1z_pos_1em05": ("", "O_g1z_pos_1em05", n_bins, -3.5, 3.5),
    "O_ka_pos_1em05":  ("", "O_ka_pos_1em05", n_bins, -3.5, 3.5),
    "O_la_pos_1em05":  ("", "O_la_pos_1em05", n_bins, -3.5, 3.5),
}
for name, config in oo_configs.items():
    analysis.add_filter(f"{name} >= {config[-2]} && {name} <= {config[-1]}", f"{name} in range")
    analysis.book_histogram_1D(name, name, config)

In [12]:
conf_3d = ("", "", n_bins, -3.5, 3.5, n_bins, -3.5, 3.5, n_bins, -3.5, 3.5)
conf_nd_sparse = ("", "", [ROOT.TAxis(n_bins,  -3.5, 3.5) for i in range(3)])
conf_nd = ("", "", 3, [n_bins]*3, [-3.5] * 3, [3.5] * 3)
analysis.book_histogram_3D("obs", conf_3d, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])
# fundamentally broken? :(
# analysis.book_histogram_ND_sparse("obs_sparse", conf_nd_sparse, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])
analysis.book_histogram_ND("obs_ND", conf_nd, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])

In [13]:
weights = [f"weight_{name}" for name in ["nominal"] + list(alt_configs.keys())]
analysis._define(("weight_nominal", "1.0"), signal_category)
analysis.book_templates(oo_configs, weights, signal_category)
analysis.book_oo_matrix(list(oo_configs.keys()))

In [14]:

# analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [15]:
for oo in oo_configs:
    analysis.book_sum(oo, oo, signal_category)
    for oo2 in oo_configs:
        analysis.define_only_on(signal_category, f"{oo}_{oo2}", f"{oo} * {oo2}")
        analysis.book_sum(f"{oo}_{oo2}", f"{oo}_{oo2}", signal_category)

In [16]:
analysis.book_reports()

In [17]:
%%time
analysis.run()

CPU times: user 21.4 s, sys: 813 ms, total: 22.2 s
Wall time: 8.82 s


In [18]:
analysis.print_reports()

       4f_sw_sl_signal             4f_sl_bkg             4f_not_sl                    2f                    3f                    5f                    6f                 higgs
        10000121 (1e+04)          606268 (1e+04)           64530 (1e+03)          716007 (2e+04)           13992 (2e+03)            4261 (1e+02)               2 (3e-01)           16222 (9e+01) All
         9902417 (1e+04)          594446 (1e+04)           64340 (1e+03)          700694 (2e+04)           13868 (2e+03)            4194 (1e+02)               2 (3e-01)           16067 (9e+01) O_g1z_pos_1em05 in range
         9879876 (1e+04)          579246 (1e+04)           64126 (1e+03)          686289 (2e+04)           13761 (2e+03)            4159 (1e+02)               2 (3e-01)           15965 (9e+01) O_ka_pos_1em05 in range
         9700683 (1e+04)          550258 (1e+04)           63697 (1e+03)          681723 (2e+04)           13703 (2e+03)            4122 (1e+02)               2 (3e-01)           15760 (9e+01

In [19]:

for name in oo_configs:
    # analysis.draw_histogram(name, categories=signal_category)
    analysis.draw_histogram(name)

In [20]:
# analysis.plot_template_bins("O_g1z_pos_1em05", alt_setup_handler, only=oo_setups)
analysis.calculate_template_parametrisation("O_g1z_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_g1z_pos_1em05", plot_path=template_par_plot_path)
analysis.calculate_template_parametrisation("O_ka_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_ka_pos_1em05", plot_path=template_par_plot_path)
analysis.calculate_template_parametrisation("O_la_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_la_pos_1em05", plot_path=template_par_plot_path)

In [21]:
# TODO: store the 3d matrix...
analysis.write_fit_inputs(list(oo_configs.keys()), histo_path)
analysis.store_raw_histograms(["obs_ND"], histo_path, opening_option="update")

In [22]:
def make_observable(h):
    res = 0.
    for i in range(h.GetNbinsX()):
        bin_content = h.GetBinContent(i+1)
        bin_center = h.GetBinCenter(i+1)
        # print(f"i: {i} bin_center: {bin_center}, bin_content: {bin_content}")
        res += bin_center * bin_content
    return res

obs_binned = []
for oo in oo_configs:
    stack, *_ = analysis.draw_histogram(oo, categories=signal_category)
    h = stack.GetHists().First()
    obs_binned.append(make_observable(h))


In [23]:
import numpy as np
obs = []
cov = []
for oo in oo_configs:
    obs.append(analysis.get_sum(oo, categories=signal_category))
    for oo2 in oo_configs:
        name = f"{oo}_{oo2}"
        cov.append(analysis.get_sum(name, categories=signal_category))

Cov = np.asarray(cov).reshape((3,3))
print(Cov)
uncut = np.asarray([7.1362e+06, 1.6882e+06, -3.90017e+06, 1.6882e+06, 4.43426e+06, 98588.2, -3.90017e+06, 98588.2, 1.36143e+07]).reshape((3,3))
print("Cov / uncut")
print(Cov / uncut)
asimov = np.asarray([4520086.37641232, 1109138.80145051, -1998222.20804277, 1109138.80145051, 3657043.91937568, 207762.12386119, -1998222.20804277, 207762.12386119, 7502627.88897558]).reshape((3,3))
print("Cov / asimov")
print(Cov / asimov)

obs_np = np.asarray(obs)
obs_binned_np = np.asarray(obs_binned)
print("obs unbinned, binned, ratio")
print(obs_np)
print(obs_binned_np)
print(obs_np / obs_binned_np)

[[ 3941018.25888312   908254.1753605  -1930641.5492786 ]
 [  908254.1753605   3009327.845014     486358.59468932]
 [-1930641.5492786    486358.59468932  7170290.07207485]]
Cov / uncut
[[0.55225726 0.53800153 0.49501472]
 [0.53800153 0.6786539  4.93323334]
 [0.49501472 4.93323334 0.52667343]]
Cov / asimov
[[0.87189003 0.81888234 0.96617961]
 [0.81888234 0.82288534 2.34093966]
 [0.96617961 2.34093966 0.95570381]]
obs unbinned, binned, ratio
[-328231.63519527 -808654.65021801 -404507.690932  ]
[-234804.14969813 -723094.33339034 -404444.88108667]
[1.39789538 1.11832525 1.0001553 ]
